# ✈️ SwiftPlan: AI Travel Concierge
## Google x Kaggle AI Agents Intensive Capstone Project

**Track:** Concierge Agents  </br>  **Author:** Ratheshan Sathiyamoorthy

---

### Project Overview
SwiftPlan is a Multi-Agent System designed to eliminate the stress of travel planning. It automates the research and scheduling process, creating personalized, budget-conscious itineraries in seconds.

### Architecture & Key Concepts
This project demonstrates three key concepts from the course:

1.  **Multi-Agent System (Sequential):** * **Research Agent:** Uses Google Search (Grounding) to find real-time data on weather, attractions, and local culture.
    * **Planner Agent:** A reasoning engine that structures the raw data into a logical day-by-day plan.
    
2.  **Tools (Hybrid):** * **Google Search:** For external knowledge.
    * **Custom Python Tool (`currency_converter`):** A defined function that the agent actively calls to calculate budgets in local currency.

3.  **Observability & Memory:** * **Session Memory:** Maintains context across the research and planning phases.
    * **Logging:** Detailed console logs track the agent's thought process, tool usage, and execution status.
  

### Diagram

```mermaid
flowchart LR
    User([User]) --> SwiftPlan{{SwiftPlan App}}
    SwiftPlan -->|1. Task: Find activities & weather| ResearchAgent[Research Agent]
    ResearchAgent -->|Search Query| GoogleSearchTool[[Google Search Tool]]
    GoogleSearchTool -->|Search Results| ResearchAgent
    ResearchAgent -->|Research Data| SwiftPlan
    SwiftPlan -->|2. Task: Create itinerary + Data| PlannerAgent[Planner Agent]
    PlannerAgent -->|Convert Budget| CurrencyConverterTool[[Currency Converter Tool]]
    CurrencyConverterTool -->|Converted Amount| PlannerAgent
    PlannerAgent -->|Generated Itinerary| FinalItinerary[(Final Itinerary)]
    FinalItinerary -->|Output| User
```

### How to Run
1.  Add your `GOOGLE_API_KEY` in the **Add-ons -> Secrets** menu.
2.  Run all cells.
3.  Use the `run_swiftplan()` function at the bottom to test different destinations!

### Setup & Configuration

In [10]:
# Installing necessary libraries
!pip install -q -U google-genai
!pip install -q rich

In [11]:
# Imports & Configuration
import os
import time
import logging
from google import genai
from google.genai import types
from rich.console import Console
from rich.markdown import Markdown

# Setup Observability (Logging) - Requirements Checklist [x]
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(message)s')
logger = logging.getLogger("SwiftPlan")

console = Console()

# Setup Client
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    GEMINI_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    client = genai.Client(api_key=GEMINI_API_KEY)
    console.print("[green]SUCCESS: API Key loaded and Client initialized![/green]")
except Exception as e:
    console.print(f"[red]ERROR: Could not load API Key. Check Add-ons -> Secrets.[/red]\n{e}")

SUCCESS: API Key loaded and Client initialized!

### Define Tools
1. Google Search: For real-time travel info.
2. Currency Converter: A custom Python function that the Planner Agent can use to estimate trip costs.

In [12]:
# Tool 1: Google Search (Built-in)
google_search_tool = types.Tool(
    google_search=types.GoogleSearch()
)

# Tool 2: Custom Currency Converter
# First, define the actual Python function
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Converts a monetary amount from one currency to another using fixed estimated rates.
    """
    rates = {
        "USD": 1.0, "EUR": 0.95, "JPY": 150.0, "GBP": 0.79, "LKR": 300.0
    }
    
    from_curr = from_currency.upper()
    to_curr = to_currency.upper()
    
    if from_curr not in rates or to_curr not in rates:
        return f"Error: Exchange rate not available for {from_curr} to {to_curr}."
    
    # Calculate
    amount_in_usd = amount / rates[from_curr]
    final_amount = amount_in_usd * rates[to_curr]
    
    return f"{amount} {from_curr} is approximately {final_amount:.2f} {to_curr}"

# Next, create the Tool object with a manual FunctionDeclaration
# We explicitly define the parameters so the model knows how to use it.
currency_tool = types.Tool(
    function_declarations=[
        types.FunctionDeclaration(
            name="currency_converter",
            description="Converts a monetary amount from one currency to another.",
            parameters=types.Schema(
                type="OBJECT",
                properties={
                    "amount": types.Schema(
                        type="NUMBER", 
                        description="The amount of money to convert"
                    ),
                    "from_currency": types.Schema(
                        type="STRING", 
                        description="The currency code to convert from (e.g. USD)"
                    ),
                    "to_currency": types.Schema(
                        type="STRING", 
                        description="The currency code to convert to (e.g. JPY)"
                    ),
                },
                required=["amount", "from_currency", "to_currency"]
            )
        )
    ]
)

console.print("[blue]Tools Configured: Google Search & Currency Converter (Schema Fixed)[/blue]")

Tools Configured: Google Search & Currency Converter (Schema Fixed)

### The Agent Class
Memory: It has self.history to remember the conversation.

Retry Logic: It automatically retries if it hits a rate limit.

Observability: It logs every step to the console.

In [13]:

class Agent:
    def __init__(self, name, model, system_instruction, tools=None, tool_map=None):
        self.name = name
        self.model = model
        self.system_instruction = system_instruction
        self.tools = tools
        self.tool_map = tool_map or {} # Dictionary mapping tool names to actual Python functions
        self.history = [] 

    def generate_response(self, prompt, context=None):
        # 1. Prepare the Initial Prompt
        full_prompt = prompt
        if context:
            full_prompt = f"PREVIOUS CONTEXT:\n{context}\n\nCURRENT TASK:\n{prompt}"

        # Add the user's prompt to history
        self.history.append(types.Content(role="user", parts=[types.Part(text=full_prompt)]))

        console.print(f"\n[bold yellow]🤖 {self.name} is working...[/bold yellow]")

        # 2. The Execution Loop (Handles Tool Calls)
        # We loop to allow multiple tool calls if needed
        max_turns = 5
        current_turn = 0
        
        while current_turn < max_turns:
            current_turn += 1
            
            try:
                # Call the Model
                config = types.GenerateContentConfig(
                    system_instruction=self.system_instruction,
                    tools=self.tools,
                    temperature=0.7
                )

                response = client.models.generate_content(
                    model=self.model,
                    contents=self.history, # Send full conversation history
                    config=config
                )

                # Check if the model returned any content
                if not response.candidates:
                    return "Error: No response from model."
                
                # Get the content part
                part = response.candidates[0].content.parts[0]
                
                # CASE A: The Model wants to call a Function (Tool Use)
                if part.function_call:
                    fc = part.function_call
                    fn_name = fc.name
                    fn_args = fc.args
                    
                    console.print(f"[dim]   ⚙️ calling tool: {fn_name}({fn_args})...[/dim]")
                    
                    # 1. Execute the Python Function
                    if fn_name in self.tool_map:
                        tool_result = self.tool_map[fn_name](**fn_args)
                    else:
                        tool_result = f"Error: Tool '{fn_name}' not found."

                    # 2. Add the Model's "Call" to history
                    self.history.append(response.candidates[0].content)

                    # 3. Add the "Result" to history (Send it back to the model)
                    # We create a FunctionResponse part
                    fn_response_part = types.Part.from_function_response(
                        name=fn_name,
                        response={"result": tool_result}
                    )
                    self.history.append(types.Content(role="user", parts=[fn_response_part]))
                    
                    # Loop continues... model will now read the result and generate text.
                    continue

                # CASE B: The Model returned Text (Final Answer)
                else:
                    response_text = part.text
                    # Add final answer to history
                    self.history.append(response.candidates[0].content)
                    
                    logger.info(f"Agent {self.name} finished successfully.")
                    return response_text

            except Exception as e:
                console.print(f"[red]Error in {self.name}: {e}[/red]")
                # If rate limit, wait and retry (Simple backoff)
                if "429" in str(e):
                    time.sleep(10)
                    continue
                return f"Error encountered: {e}"
        
        return "Error: Maximum tool turns exceeded."

### Instantiate the Agents

1. Research Agent: Uses Google Search to find data.

2. Planner Agent: Takes that data and builds the schedule (No search tool, just pure reasoning).

In [14]:
# Instantiate Agents

MODEL_ID = "gemini-2.5-flash-lite" 

# 1. Research Agent
researcher = Agent(
    name="Research Agent",
    model=MODEL_ID,
    system_instruction="""
    You are a Travel Researcher.
    1. Use Google Search to find real-time info.
    2. Focus on: Top 3 attractions, 1 hidden gem, and local food.
    3. Return a detailed, factual list.
    """,
    tools=[google_search_tool],
    tool_map={} # Researcher uses built-in Google Search, no custom map needed
)

# 2. Planner Agent
# We create a map so the Agent knows which function to run
tool_mapping = {
    "currency_converter": currency_converter
}

planner = Agent(
    name="Planner Agent",
    model=MODEL_ID,
    system_instruction="""
    You are a Trip Planner.
    1. Create a day-by-day itinerary based on the research provided.
    2. ALWAYS use the 'currency_converter' tool to show the budget in local currency.
    3. Format the final output in clean Markdown.
    """,
    tools=[currency_tool],
    tool_map=tool_mapping # <--- We pass the map here!
)

console.print(f"[green]Agents Ready & Linked![/green]")

Agents Ready & Linked!

### Session Management and Orchestration

In [19]:
# The "SwiftPlan" Workflow (Orchestration)

def run_swiftplan(destination, duration, interests, budget_usd):
    console.print(f"[bold cyan]✈️ SwiftPlan: {destination} ({duration})[/bold cyan]")
    
    # --- Step 1: Research Phase ---
    console.print("[bold]1️⃣  Starting Research...[/bold]")
    task1 = f"Find top things to do in {destination} for someone interested in {interests}. Include specific details like location names and what makes them special. Also, check the general weather forecast for the next 7 days."
    research_data = researcher.generate_response(task1)
    
    time.sleep(1)

    
    
    # --- Step 2: Pre-calculate Budget ---
    console.print("[bold]2️⃣  Calculating Budget...[/bold]")
    # We manually call the currency converter function here.
    # For this demo, we assume the destination is Tokyo and hardcode 'JPY'.
    # In a real app, you could use another tool to find the currency code first.
    local_currency_code = "JPY" 
    converted_budget_str = currency_converter(budget_usd, "USD", local_currency_code)
    console.print(f"[dim]   💰 Pre-calculated budget: {converted_budget_str}[/dim]")

    

    # --- Step 3: Planning Phase ---
    console.print("[bold]3️⃣  Generating Itinerary...[/bold]")
    # We now give the planner the research AND the already-converted budget.
    task2 = f"""
    Create a detailed {duration} day-by-day itinerary for {destination}.
    
    Here is the converted budget you must work with: {converted_budget_str}.
    
    Your goal is to plan activities and meals that realistically fit within this total budget. 
    Mention estimated costs for items in the itinerary to show how the budget is being used.
    
    Format the output in clean, easy-to-read Markdown with clear headings for each day.
    """
    
    # Pass the research data as context
    final_plan = planner.generate_response(task2, context=research_data)
    
    # --- Step 4: Output ---
    console.print("\n" + "="*50)
    console.print(f"[bold green]✅ ITINERARY FOR {destination.upper()}[/bold green]")
    console.print("="*50 + "\n")
    console.print(Markdown(final_plan))

### Testing

In [20]:
# === TEST RUN ===
destination = "Tokyo, Japan"
duration = "3 Days"
interests = "Tech, Anime, and Shushi"
budget_usd = 500

# execute
run_swiftplan(destination, duration, interests, budget_usd)

✈️ SwiftPlan: Tokyo, Japan (3 Days)

1️⃣  Starting Research...

🤖 Research Agent is working...

2025-12-01 07:28:23,150 - google_genai.models - AFC is enabled with max remote calls: 10.
2025-12-01 07:28:30,022 - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-12-01 07:28:30,027 - SwiftPlan - Agent Research Agent finished successfully.


2️⃣  Calculating Budget...

...

2025-12-01 07:28:41,042 - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-12-01 07:28:41,046 - SwiftPlan - Agent Planner Agent finished successfully.


==================================================

✅ ITINERARY FOR TOKYO, JAPAN

==================================================

Tokyo 3-Day Itinerary: Tech, Anime, and Sushi Adventure                              

This itinerary balances your interests in technology, anime, and sushi within a budget of approximately 75,000 JPY.
Costs are estimates and can vary.                                                                                  

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                     Day 1: Akihabara Immersion & Retro Gaming                                     

 • Morning (9:00 AM - 12:00 PM): Akihabara Exploration                                                             
    • Dive into the heart of Akihabara Electric Town. Explore multi-story electronics stores like Yodobashi Camera 
      for the latest gadgets and anime/manga shops such as Mandarake Complex and Animate for collectibles.         
    • Estimated Cost: Free to explore, but budget for any purchases (e.g., souvenirs, small electronics). Let's    
      allocate 5,000 JPY for initial exploration/small buys.                                                       
 • Lunch (12:00 PM - 1:00 PM): Themed Cafe Experience                                                              
    • Enjoy lunch at one of Akihabara's unique themed cafes, such as a maid cafe or an anime-themed cafe.          
    • Estimated Cost: 2,500 JPY                                                                                    
 • Afternoon (1:00 PM - 5:00 PM): Arcades & Hidden Gems                                                            
    • Experience the vibrant arcades like Sega Akihabara or Taito Station. Afterward, head to Nakano Broadway (a   
      short train ride away) for a more focused anime and manga collectible hunt.                                  
    • Estimated Cost: 3,000 JPY for arcade games and train fare to Nakano.                                         
 • Dinner (6:00 PM - 7:30 PM): Casual Sushi Dinner                                                                 
    • Find a local sushi restaurant in Akihabara or near your accommodation for a delicious and moderately priced  
      sushi dinner.                                                                                                
    • Estimated Cost: 4,000 JPY                                                                                    
 • Evening (7:30 PM onwards): 8 Bit Café                                                                           
    • Head to Shinjuku to experience the nostalgic 8 Bit Café. Enjoy drinks and play classic video games.          
    • Estimated Cost: 3,000 JPY (for drinks and game time)                                                         

Day 1 Estimated Total: 17,500 JPY                                                                                  

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                         Day 2: Futuristic Art & Bay Views                                         

 • Morning (10:00 AM - 1:00 PM): teamLab Planets TOKYO                                                             
    • Immerse yourself in the digital art wonderland of teamLab Planets in Toyosu. Book your tickets in advance    
      online to save time.                                                                                         
    • Estimated Cost: 3,800 JPY (entry ticket) + 500 JPY (train fare)                                              
 • Lunch (1:00 PM - 2:00 PM): Odaiba Waterfront Dining                                                             
    • Travel to Odaiba and have lunch at one of the many restaurants offering views of Tokyo Bay.                  
    • Estimated Cost: 3,000 JPY                                                                                    
 • Afternoon (2:00 PM - 5:00 PM): Odaiba Exploration                            

In [21]:
# === TEST RUN 2 ===
destination = "Kandy, Sri Lanka"
duration = "4 Days"
interests = "History, Nature, and Fried Rice"
budget_usd = 500

# execute
run_swiftplan(destination, duration, interests, budget_usd)

✈️ SwiftPlan: Kandy, Sri Lanka (4 Days)

1️⃣  Starting Research...

🤖 Research Agent is working...

2025-12-01 07:31:48,638 - google_genai.models - AFC is enabled with max remote calls: 10.
2025-12-01 07:31:52,457 - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-12-01 07:31:52,460 - SwiftPlan - Agent Research Agent finished successfully.


2️⃣  Calculating Budget...

...

2025-12-01 07:32:02,914 - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-12-01 07:32:02,917 - SwiftPlan - Agent Planner Agent finished successfully.


==================================================

✅ ITINERARY FOR KANDY, SRI LANKA

==================================================

It seems there was a small mix-up in the currency conversion for your Kandy trip. The budget of 500 USD is         
approximately 160,000 LKR (Sri Lankan Rupees), not 75,000 JPY (Japanese Yen). I will create your 4-day itinerary   
for Kandy using the correct budget of 160,000 LKR.                                                                 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                             Kandy 4-Day Itinerary: History, Nature, and Local Flavors                             

This itinerary is designed to explore the historical, natural, and culinary highlights of Kandy within your budget 
of approximately 160,000 LKR. Costs are estimates and can vary based on your choices and negotiation skills.       

                                        Day 1: Sacred Sites and City Views                                         

 • Morning (9:00 AM - 12:00 PM): Temple of the Sacred Tooth Relic                                                  
    • Begin your exploration at the revered Temple of the Sacred Tooth Relic (Sri Dalada Maligawa). Allow ample    
      time to soak in the spiritual atmosphere and admire the intricate architecture. Dress modestly (shoulders and
      knees covered).                                                                                              
    • Estimated Cost: Entrance fee is approximately 1,500 LKR.                                                     
 • Lunch (12:30 PM - 1:30 PM): Local Fried Rice Experience                                                         
    • Enjoy a delicious and authentic Sri Lankan fried rice dish at a local restaurant near the temple. Many small 
      eateries offer flavorful options.                                                                            
    • Estimated Cost: 1,000 LKR                                                                                    
 • Afternoon (2:00 PM - 4:00 PM): Kandy Lake & Bahirawakanda Statue                                                
    • Take a leisurely stroll around the serene Kandy Lake (Bogambara Lake), enjoying the city and temple views.   
      Afterward, hire a tuk-tuk to visit the Bahirawakanda Vihara Buddha Statue for panoramic vistas of Kandy.     
    • Estimated Cost: Tuk-tuk hire (round trip) approx. 1,000 LKR.                                                 
 • Evening (6:00 PM - 7:30 PM): Dinner with a View                                                                 
    • Dine at a restaurant overlooking Kandy Lake or in the city center, perhaps trying some local curries.        
    • Estimated Cost: 2,500 LKR                                                                                    

Day 1 Estimated Total: 6,000 LKR                                                                                   

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                    Day 2: Botanical Wonders & Ancient Temples                                     

 • Morning (9:00 AM - 12:30 PM): Royal Botanic Gardens, Peradeniya                                                 
    • Take a tuk-tuk or taxi to the magnificent Royal Botanic Gardens in Peradeniya. Spend your morning exploring  
      the vast collections of plants, including the famous orchid house and spice garden.                          
    • Estimated Cost: Tuk-tuk/taxi (round trip) approx. 2,000 LKR. Entrance fee approx. 2,000 LKR.                 
 • Lunch (1:00 PM - 2:00 PM): Garden Cafe or Local Eatery                                                          
    • Have a relaxed lunch at the cafe within the gardens or find a local spot back towards Kandy.                 
    • Estimated Cost: 1,500 LKR                                                                                    
 • Afternoon (2:30 PM - 5:30 PM): The Three Temp